# BGSA - Images (e.g. JPG)

****
### Code starts here

In [ ]:
import os
import numpy as np
from PIL import Image 
from scipy.misc import *
#from argparse import ArgumentParser
from bgsa_utils import *

from ipywidgets import widgets
from IPython.display import FileLink

import matplotlib.pyplot as plt
%matplotlib inline

In [ ]:
uploader = widgets.FileUpload()

In [ ]:
def fun_proc(inputBrightness,inputContrast,inputHue,inputSaturation):
  print('Processing...')

  #get default parameter values
  brightness = int(inputBrightness.value)
  contrast = int(inputContrast.value)
  hue = int(inputHue.value)
  saturation = float(inputSaturation.value)

  #default file
  file_path = 'gland_86_2.jpg'

  # if image was uploaded
  #if bool(uploader.value):
  #  uploaded_filename = next(iter(uploader.value))
  #  content = uploader.value[uploaded_filename]['content']
  #  with open('myfile', 'wb') as f: f.write(content)
  #  file_path = './myfile'

  im = Image.open(os.path.join(os.getcwd(), file_path), mode='r')  
  red_im = get_red(im, brightness, contrast)
  brown_im = get_brown(im, brightness, contrast, hue, saturation)

  brown_pix = get_white_pixels(brown_im)
  red_pix = get_white_pixels(red_im)

  print("Set parameters:")
  print("Brightness "+str(brightness))
  print("contrast "+str(contrast))
  print("hue "+str(hue))
  print("saturation "+str(saturation))
    
  print("\n")
  print("Results:")
  print("brown pixels "+str(brown_pix))
  print("red pixels "+str(red_pix))
  
  fig, ax = plt.subplots(1,3, figsize=(20, 20))
  ax[0].imshow(im)
  ax[0].axis('off')
  ax[0].set_title("Original image")
  ax[1].imshow(brown_im)
  ax[1].set_title("Brown mask")
  ax[2].imshow(red_im)
  ax[2].set_title("Red mask")
  
  brown_im.save(file_path + "_brown.png", "PNG")
  red_im.save(file_path + "_red.png", "PNG")

  print("\n")
  print("Links to download mask images: ")
  display(FileLink("./" + file_path + "_brown.png"))
  display(FileLink("./" + file_path + "_red.png"))


In [ ]:
def pseudo_GUI():
    # displaying the text widget
    inputBrightness = widgets.Text(description="Brightness", width=50, value=str(-100))
    display(inputBrightness)

    inputContrast = widgets.Text(description="Contrast", width=50, value=str(300))
    display(inputContrast)

    inputHue = widgets.Text(description="Hue", width=50, value=str(-90))
    display(inputHue)

    inputSaturation = widgets.Text(description="Saturation", width=50, value=str(0.65))
    display(inputSaturation)

    button = widgets.Button(description = 'Calculate!')
    display(button)

    def on_button_clicked(self):
        fun_proc(inputBrightness,inputContrast,inputHue,inputSaturation)
    button.on_click(on_button_clicked)

### Code ends here
****

### Optional upload (single image, max. 10MB):

If there is no upload, use default test image.

In [ ]:
uploader

## BGSA method pseudo-GUI
Set values of method parameters (default preloaded).
Click 'Calculate!' button to begin processing.


In [ ]:
pseudo_GUI()


# BGSA - NDPI example

### Parameters of WSI loading

In [ ]:
split = 2
level = 3

### Load example WSI and process as a whole

In [ ]:
#@title
from openslide import OpenSlide
import zipfile

!wget http://openslide.cs.cmu.edu/download/openslide-testdata/Mirax/CMU-1-Saved-1_16.zip

with zipfile.ZipFile("CMU-1-Saved-1_16.zip","r") as zip_ref:
    zip_ref.extractall("test_mrxs")


ndpi         = OpenSlide('./test_mrxs/CMU-1-Saved-1_16.mrxs')
ndpi_width   = ndpi.dimensions[0]
ndpi_height  = ndpi.dimensions[1]
total_width  = ndpi.level_dimensions[level][0]
total_height = ndpi.level_dimensions[level][1]


x     = 0
y     = 0
w     = round(total_width-1)
h     = round(total_height-1)

region  = ndpi.read_region((x,y), level, (w, h))
red     = get_red(region)
brown   = get_brown(region)


fig, ax = plt.subplots(1,3, figsize=(20, 20))
ax[0].imshow(region)
ax[0].axis('off')
ax[0].set_title("Original image")
ax[1].imshow(brown)
ax[1].set_title("Brown mask")
ax[2].imshow(red)
ax[2].set_title("Red mask")


### Show split of example WSI and numerical result

In [ ]:
red_sum     = 0.0
brown_sum   = 0.0
surface_sum = 0.0

fig, axs = plt.subplots(split,split)
for i in range(split):
    for j in range(split):
        x     = i * round(ndpi_width / split)
        y     = j * round(ndpi_height / split)
        w     = round(total_width / split)
        h     = round(total_height / split)
        
        region  = ndpi.read_region((x,y), level, (w, h))
        #print(str(i)+' '+str(j))
        axs[i, j].imshow(region)
        red     = get_red(region)
        brown   = get_brown(region)

        red_sum    += get_white_pixels(red)
        brown_sum  += get_white_pixels(brown)   

print("Sum of red pixels: " + str(red_sum))
print("Sum of brown pixels: "  + str(brown_sum))

print("\n")
print("Split WSI: " )